In [4]:
import os
import time
from datetime import timedelta
from pytz import timezone
import pytz
from datetime import datetime

import cv2
import numpy as np
import streamlink
from imageai.Detection import VideoObjectDetection, ObjectDetection

from selenium import webdriver
import csv
import pandas as pd


def crop_frame(frame, target_img_name, y1=150, y2=500, x1=0, x2=1000):
    """
    only crop frame to evaluate the baseline. not a offical function, therefore outsied the class. will be deleted after evaluation.
    """
    frame = frame[y1:y2, x1:x2]

    if not os.path.isdir('.\\dublin_night_baseline'):
        os.makedirs('.\\dublin_night_baseline')
    path = '.\\dublin_night_baseline'
    cv2.imwrite(os.path.join(path, target_img_name), frame)
    return frame


######################################################################################################
#list of webcam
london = 'https://www.earthcam.com/world/england/london/abbeyroad/?cam=abbeyroad_uk'
timesquare = 'https://www.earthcam.com/usa/newyork/timessquare/?cam=tsrobo1'
dublin = 'https://www.earthcam.com/world/ireland/dublin/?cam=templebar'
######################################################################################################
#list of timezone
Dublin = 'Europe/Dublin'
London = 'Europe/London'
NYC = 'America/New_York'
######################################################################################################


class CountingObject(object):
    """
    A class of counting objects
    """

    algos = {
        "resnet": "resnet50_coco_best_v2.0.1.h5",
        "yolov3": "yolo.h5",
        "yolo_tiny": "yolo-tiny.h5"
    }

    def __init__(self, stream_link):
        self.stream_link = stream_link
        self.streams = streamlink.streams(stream_link)
        if self.streams is None:
            raise ValueError("cannot open the stream link %s" % stream_link)

        q = list(self.streams.keys())[0]
        self.stream = self.streams['%s' % q]

        self.target_img_path = os.getcwd()

        self.detector = ObjectDetection()
        if self.detector is None:
            raise ValueError("Detector of objects is None")

    def detector_init(self, algo="resnet", speed="nomal"):
        """
        Must be invoked after instantiate for initialize a object detector. 
        
        Args:
            algo (str): The algorithm of object detection tasks. "resnet"(default), "yolov3", "yolo_tiny".
            speed (str): The detection speed for object detetion tasks. "normal"(default), "fast", "faster" , "fastest" and "flash".
        
        Returns:
            void
        
        """

        if algo == "resnet":
            self.detector.setModelTypeAsRetinaNet()
            self.detector.setModelPath(
                os.path.join(self.target_img_path, self.algos["resnet"]))
        elif algo == "yolov3":
            self.detector.setModelTypeAsYOLOv3()
            self.detector.setModelPath(
                os.path.join(self.target_img_path, self.algos["yolov3"]))
        elif algo == "yolo_tiny":
            self.detector.setModelTypeAsTinyYOLOv3()
            self.detector.setModelPath(
                os.path.join(self.target_img_path, self.algos["yolo_tiny"]))
        else:
            print("Given algorithm of object detection is invalid.")
            return

        self.detector.loadModel(detection_speed=speed)
        self.custom_objects = self.detector.CustomObjects(person=True)

    def put_text_to_img(self,
                        img,
                        text,
                        pos=(50, 50),
                        fontColor=(0, 0, 255),
                        lineType=2):
        """
        Put text to an image.
        
        Args:
            img : An image represented by numpy array. You can use cv2.imread(path_to_iamge) to read an image in the filesystem by
                    giving the image path.
            text (str): The text what you want to put to the image.
            pos(tuple): x and y position relative to the origin (0,0) at the top left.
            fontColor(tuple): R G B channel.
            lineType(int): Type of line.
        
        Returns:
            void
        
        """
        
        if img is None:
            print("Put text to a none image.")
            return

        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 1

        cv2.putText(img, text, pos, font, fontScale, fontColor, lineType)

    def capture_frame_by_stream_wrapper(self,
                                        image_prefix="stream",
                                        mprob=30,
                                        num_im=6,
                                        time_interval=10,
                                        tz=None):
        """
        A wrapper of the function capture_frame_by_stream.
        
        Args:
            image_prefix (str): Prefix of target images. The postfix is numerated by numbers.
            mprob (int): Minimum probability to be a person.
            num_im(int): How many images will be taken.
            time_interval(int): Time interval of taking next image, the unit is second.
			tz: Time zone
        
        Returns:
            void
        
        """
        
        print(
            "The current conuting function is based on capture frame by stream."
        )

        dir_path = os.path.join(self.target_img_path, image_prefix)
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)
        frames_res = []
        if num_im <= 0:
            try:
                i = 0
                while True:
                    i = i + 1
                    frame_res = self.capture_frame_by_stream(
                        image_prefix, i, mprob, tz)
                    frames.res.append(frame_res)
                    time.sleep(time_interval)
            except KeyboardInterrupt:
                return frames_res
                print('Abort by key interrupt.')
        else:
            for i in range(num_im):
                frame_res = self.capture_frame_by_stream(
                    image_prefix, i, mprob, tz)
                frames_res.append(frame_res)
                time.sleep(time_interval)

            return frames_res

    def capture_frame_by_stream(self,
                                image_prefix="stream",
                                image_index=0,
                                mprob=30,
                                tz=None) -> int:
        """
        capture a frame from a online stream, namely webcam.
        
        Args:
            image_prefix (str): Prefix of target images. The postfix is numerated by numbers.
            image_index (int): The postfix of target images. By default, numerated from 0.
            mprob(int): Minimum probability to be a person.
			tz: Time zone
		
        Returns:
            dict: The name of target image and The number of persons in an image counted by the model.
        """
        
        video_cap = cv2.VideoCapture(self.stream.url)
        dir_path = os.path.join(self.target_img_path, image_prefix)

        if video_cap is None:
            print("Open webcam [%s] failed." % self.stream.url)
            return None
        else:
            ret, frame = video_cap.read()

            if not ret:
                print("Captured frame is broken.")
                video_cap.release()
                return None
            else:
                print("-----------------------------------------------------")
                

                if tz is None:
                    current_time = datetime.utcnow().strftime(
                        "%a %Y-%m-%d %H:%M:%S")
                    print('### time zone is None, therefore use utc time###')
                else:
                    current_time = datetime.now(
                        timezone(tz)).strftime("%a %Y-%m-%d %H:%M:%S")

                print("Capturing frame %d." % image_index)
                target_img_name = "{}{}.png".format(image_prefix, image_index)
                frame = crop_frame(
                    frame,
                    target_img_name)  # comment to unuse the crop function.
                
                cv2.imwrite(os.path.join(dir_path, target_img_name), frame)

                detections = self.detector.detectCustomObjectsFromImage(
                    custom_objects=self.custom_objects,
                    input_image=os.path.join(dir_path, target_img_name),
                    output_image_path=os.path.join(dir_path, target_img_name),
                    minimum_percentage_probability=mprob)

                print(
                    "The number of person in frame %d (%s):" %
                    (image_index, target_img_name), len(detections))
                print(
                    "The current time in frame %d (%s):" %
                    (image_index, target_img_name), current_time)

                img = cv2.imread(os.path.join(dir_path, target_img_name))
                # put the number of persons to the image and put timestamp to the image
                self.put_text_to_img(
                    img, "The number of person:%s " % str(len(detections)))
                self.put_text_to_img(
                    img, "The current time:%s " % current_time, pos=(50, 300))

                cv2.imwrite(os.path.join(dir_path, target_img_name), img)
                video_cap.release()

                return target_img_name, len(detections), current_time

    def capture_frame_by_screenshot_wrapper(self,
                                            image_prefix="screenshot",
                                            mprob=30,
                                            num_im=6,
                                            time_interval=10,
                                            tz=None):
        """
        A wrapper of the function capture_frame_by_screenshot.
        
        Args:
            image_prefix (str): Prefix of target images. The postfix is numerated by numbers.
            mprob (int): Minimum probability to be a person.
            num_im(int): How many images will be taken.
            time_interval(int): Time interval of taking next image, the unit is second.
			tz: Time zone.
        
        Returns:
            void
        
        """
        
        print(
            "The current conuting function is based on capture frame by screenshot."
        )

        frames_res = []
        dir_path = os.path.join(self.target_img_path, image_prefix)
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)
        if num_im <= 0:
            try:
                i = 0
                while True:
                    i = i + 1
                    frame_res = self.capture_frame_by_screenshot(
                        image_prefix, i, mprob, tz)
                    frames_res.append(frame_res)
                    time.sleep(time_interval)
            except KeyboardInterrupt:
                if self.driver is not None:
                    self.driver.quit()
                return frames_res
                print('Abort by key interrupt.')
        else:
            for i in range(num_im):
                frame_res = self.capture_frame_by_screenshot(
                    image_prefix, i, mprob, tz)
                frames_res.append(frame_res)
                time.sleep(time_interval)

            if self.driver is not None:
                self.driver.quit()

            return frames_res

    def capture_frame_by_screenshot(self,
                                    image_prefix="screenshot",
                                    image_index=0,
                                    mprob=30,
                                    num_im=6,
                                    tz=None) -> int:
        """
       capture an image by taking a screenshot on an opened website via browser.
        
        Args:
            image_prefix (str): Prefix of target images. The postfix is numerated by numbers.
            image_index (int): The postfix of target images. By default, numerated from 0.
            mprob(int): Minimum probability to be a person.
			tz: Time zone.
        
        Returns:
            dict: The name of target image and The number of persons in an image counted by the model.
        
        """
        
        dir_path = os.path.join(self.target_img_path, image_prefix)

        if self.driver is None:
            print("Web driver is none.")
            return None
        else:
            print("-----------------------------------------------------")

            if tz is None:
                current_time = datetime.utcnow().strftime(
                    "%a %Y-%m-%d %H:%M:%S")
                print('### time zone is None, therefore use utc time###')
            else:
                current_time = datetime.now(
                    timezone(tz)).strftime("%a %Y-%m-%d %H:%M:%S")

            target_img_name = "{}{}.png".format(image_prefix, image_index)
            print("Taking screenshot %d..." % image_index)
            self.driver.save_screenshot(
                os.path.join(dir_path, target_img_name))
            detections = self.detector.detectCustomObjectsFromImage(
                custom_objects=self.custom_objects,
                input_image=os.path.join(dir_path,
                                         target_img_name),
                output_image_path=os.path.join(dir_path, target_img_name),
                minimum_percentage_probability=mprob)

            print(
                "The number of person in frame %d (%s):" % (image_index,
                                                            target_img_name),
                len(detections))
            print(
                "The current time in frame %d (%s):" %
                (image_index, target_img_name), current_time)

            img = cv2.imread(os.path.join(dir_path, target_img_name))
            # put the number of persons to the image
            self.put_text_to_img(
                img, "The number of person is:%s" % str(len(detections)))
            self.put_text_to_img(
                img, "The current time:%s " % current_time, pos=(50, 300))

            cv2.imwrite(os.path.join(dir_path, target_img_name), img)

            return target_img_name, len(detections), current_time

    def init_webdriver(self):
        """
       Initialize the webdriver of Chrome by using the python lib selenium.
        
        Args:
            Void
        
        Returns:
            Void
        """
        
        self.driver = webdriver.Chrome(
        )  # Optional argument, if not specified will search path.
        self.driver.get(self.stream_link)
        time.sleep(15)  # Jump over the ads
        
    def stroe_info_in_df_csv(self, image_prefix="counting_person"):
        """
       Collect test dataset by storing the image name and the detected number of persons in a csv file.
        
        Args:
            info():  
        
        Returns:
            Void
        
        """  
        df = pd.DataFrame(
            np.array(res), columns=['image_name', 'detected_num', 'time'])
        df["counted_num"] = ""  #only for baseline
        df.to_csv(
            path_or_buf=os.path.join(self.target_img_path, "%s.csv" %
                                     image_prefix))
        return df

if __name__ == "__main__":
    #     scheduler = BlockingScheduler()
    print("Starting...")
    counting_person = CountingObject(dublin)
    counting_person.detector_init()

    by_stream_flag = True
    img_prefix = "dublin_night"
    res = []
    if by_stream_flag:
        res = counting_person.capture_frame_by_stream_wrapper(
            image_prefix=img_prefix, num_im=50, time_interval=180, tz=Dublin)

    else:
        counting_person.init_webdriver()
        res = counting_person.capture_frame_by_screenshot_wrapper(num_im=2)

#     counting_person.store_baseline_info_in_csv(res)
    df = counting_person.stroe_info_in_df_csv(image_prefix=img_prefix)
    display(df)

    print('###Exit...')

Starting...
The current conuting function is based on capture frame by stream.
-----------------------------------------------------
Capturing frame 0.
The number of person in frame 0 (dublin_night0.png): 19
The current time in frame 0 (dublin_night0.png): Thu 2018-12-27 21:15:12
-----------------------------------------------------
Capturing frame 1.
The number of person in frame 1 (dublin_night1.png): 18
The current time in frame 1 (dublin_night1.png): Thu 2018-12-27 21:18:23
-----------------------------------------------------
Capturing frame 2.
The number of person in frame 2 (dublin_night2.png): 21
The current time in frame 2 (dublin_night2.png): Thu 2018-12-27 21:21:31
-----------------------------------------------------
Capturing frame 3.
The number of person in frame 3 (dublin_night3.png): 23
The current time in frame 3 (dublin_night3.png): Thu 2018-12-27 21:24:39
-----------------------------------------------------
Capturing frame 4.
The number of person in frame 4 (dublin_

-----------------------------------------------------
Capturing frame 40.
The number of person in frame 40 (dublin_night40.png): 13
The current time in frame 40 (dublin_night40.png): Thu 2018-12-27 23:19:23
-----------------------------------------------------
Capturing frame 41.
The number of person in frame 41 (dublin_night41.png): 14
The current time in frame 41 (dublin_night41.png): Thu 2018-12-27 23:22:28
-----------------------------------------------------
Capturing frame 42.
The number of person in frame 42 (dublin_night42.png): 14
The current time in frame 42 (dublin_night42.png): Thu 2018-12-27 23:25:34
-----------------------------------------------------
Capturing frame 43.
The number of person in frame 43 (dublin_night43.png): 7
The current time in frame 43 (dublin_night43.png): Thu 2018-12-27 23:28:39
-----------------------------------------------------
Capturing frame 44.
The number of person in frame 44 (dublin_night44.png): 14
The current time in frame 44 (dublin_nigh

,image_name,detected_num,time,counted_num
0,dublin_night0.png,19,Thu 2018-12-27 21:15:12,
1,dublin_night1.png,18,Thu 2018-12-27 21:18:23,
2,dublin_night2.png,21,Thu 2018-12-27 21:21:31,
3,dublin_night3.png,23,Thu 2018-12-27 21:24:39,
4,dublin_night4.png,29,Thu 2018-12-27 21:27:47,
5,dublin_night5.png,22,Thu 2018-12-27 21:30:53,
6,dublin_night6.png,17,Thu 2018-12-27 21:33:59,
7,dublin_night7.png,17,Thu 2018-12-27 21:37:05,
8,dublin_night8.png,21,Thu 2018-12-27 21:40:10,
9,dublin_night9.png,15,Thu 2018-12-27 21:43:16,


###Exit...


In [5]:
def crop_frame(frame, target_img_name, y1=150, y2=500, x1=0, x2=1000):
    """
    only crop frame to evaluate the baseline. not a offical function, therefore outsied the class. will be deleted after evaluation.
    """
    frame = frame[y1:y2, x1:x2]

    if not os.path.isdir('.\\dublin_day_baseline'):
        os.makedirs('.\\dublin_day_baseline')
    path = '.\\dublin_day_baseline'
    cv2.imwrite(os.path.join(path, target_img_name), frame)
    return frame

if __name__ == "__main__":
    #     scheduler = BlockingScheduler()
    print("Starting...")
    counting_person = CountingObject(dublin)
    counting_person.detector_init()

    by_stream_flag = True
    img_prefix = "dublin_day"
    res = []
    if by_stream_flag:
        res = counting_person.capture_frame_by_stream_wrapper(
            image_prefix=img_prefix, num_im=50, time_interval=180, tz=Dublin)

    else:
        counting_person.init_webdriver()
        res = counting_person.capture_frame_by_screenshot_wrapper(num_im=2)

#     counting_person.store_baseline_info_in_csv(res)
    df = counting_person.stroe_info_in_df_csv(image_prefix=img_prefix)
    display(df)

    print('###Exit...')

Starting...
The current conuting function is based on capture frame by stream.
-----------------------------------------------------
Capturing frame 0.
The number of person in frame 0 (dublin_day0.png): 3
The current time in frame 0 (dublin_day0.png): Fri 2018-12-28 10:15:08
-----------------------------------------------------
Capturing frame 1.
The number of person in frame 1 (dublin_day1.png): 20
The current time in frame 1 (dublin_day1.png): Fri 2018-12-28 10:18:18
-----------------------------------------------------
Capturing frame 2.
The number of person in frame 2 (dublin_day2.png): 3
The current time in frame 2 (dublin_day2.png): Fri 2018-12-28 10:21:23
-----------------------------------------------------
Capturing frame 3.
The number of person in frame 3 (dublin_day3.png): 1
The current time in frame 3 (dublin_day3.png): Fri 2018-12-28 10:24:29
-----------------------------------------------------
Capturing frame 4.
The number of person in frame 4 (dublin_day4.png): 3
The cu

The number of person in frame 40 (dublin_day40.png): 10
The current time in frame 40 (dublin_day40.png): Fri 2018-12-28 12:19:08
-----------------------------------------------------
Capturing frame 41.
The number of person in frame 41 (dublin_day41.png): 19
The current time in frame 41 (dublin_day41.png): Fri 2018-12-28 12:22:14
-----------------------------------------------------
Capturing frame 42.
The number of person in frame 42 (dublin_day42.png): 26
The current time in frame 42 (dublin_day42.png): Fri 2018-12-28 12:25:20
-----------------------------------------------------
Capturing frame 43.
The number of person in frame 43 (dublin_day43.png): 29
The current time in frame 43 (dublin_day43.png): Fri 2018-12-28 12:28:26
-----------------------------------------------------
Capturing frame 44.
The number of person in frame 44 (dublin_day44.png): 17
The current time in frame 44 (dublin_day44.png): Fri 2018-12-28 12:31:32
-----------------------------------------------------
Captu

,image_name,detected_num,time,counted_num
0,dublin_day0.png,3,Fri 2018-12-28 10:15:08,
1,dublin_day1.png,20,Fri 2018-12-28 10:18:18,
2,dublin_day2.png,3,Fri 2018-12-28 10:21:23,
3,dublin_day3.png,1,Fri 2018-12-28 10:24:29,
4,dublin_day4.png,3,Fri 2018-12-28 10:27:35,
5,dublin_day5.png,3,Fri 2018-12-28 10:30:40,
6,dublin_day6.png,2,Fri 2018-12-28 10:33:46,
7,dublin_day7.png,4,Fri 2018-12-28 10:36:52,
8,dublin_day8.png,4,Fri 2018-12-28 10:39:57,
9,dublin_day9.png,7,Fri 2018-12-28 10:43:03,


###Exit...
